## Preprocess raw images and store the processed images in the spatialdata.images container with '_preprocessed' suffix

### Define libraries, paths and parameters

In [10]:
# Libraries
import spatialdata as sd
import os
import yaml
# from functions.processing_utils import *
import numpy as np
import cv2
from functions.processing_utils import *
from dask_image.ndfilters import maximum_filter, minimum_filter
import dask.array as da 
from functions.utils import remove_small_objects_dask
from spatialdata.models import Image2DModel, Labels2DModel, Labels3DModel

In [4]:
#Paths
sample_type = 'Flox_Control'
data_dir = '/srv/data/michielvc/data/CSF1_ligand_data/SpatialData/'

# open config file and extract relevant information
config = yaml.safe_load(open(data_dir + f'{sample_type}_config.yaml'))
file_names = config['file_names']

# Parameters
minmax_filter_size = tuple(config['minmax_filter_size'])
contrast_clip = 3.5
clahe = cv2.createCLAHE(clipLimit=contrast_clip, tileGridSize=(8, 8))
# artifact_size_threshold = config['artifact_size_threshold']
artifact_size_threshold = 250

dapi_ch = config['marker_channel']['DAPI']

### Load data as SpatialData Object

In [5]:
sdata = sd.read_zarr(os.path.join(data_dir, f'{sample_type}.sd'))
sdata

SpatialData object with:
├── Images
│     ├── 'E2M1FloxControl_preprocessed': SpatialImage[cyx] (5, 7533, 16659)
│     ├── 'E2M1FloxControl_raw': SpatialImage[cyx] (5, 7533, 16659)
│     ├── 'E2M3FloxControl_preprocessed': SpatialImage[cyx] (5, 11178, 18468)
│     ├── 'E2M3FloxControl_raw': SpatialImage[cyx] (5, 11178, 18468)
│     ├── 'E2M5FloxControl_preprocessed': SpatialImage[cyx] (5, 9342, 12987)
│     ├── 'E2M5FloxControl_raw': SpatialImage[cyx] (5, 9342, 12987)
│     ├── 'E2M6FloxControl_preprocessed': SpatialImage[cyx] (5, 11205, 14823)
│     ├── 'E2M6FloxControl_raw': SpatialImage[cyx] (5, 11205, 14823)
│     ├── 'E2M7FloxControl_preprocessed': SpatialImage[cyx] (5, 9342, 16632)
│     ├── 'E2M7FloxControl_raw': SpatialImage[cyx] (5, 9342, 16632)
│     ├── 'E2M8FloxControl_preprocessed': SpatialImage[cyx] (5, 7533, 14823)
│     ├── 'E2M8FloxControl_raw': SpatialImage[cyx] (5, 7533, 14823)
│     ├── 'E2M15FloxControl_preprocessed': SpatialImage[cyx] (5, 9342, 14823)
│     ├── 'E

### Perform preprocessing and save preprocessed image in same SpatialData object
Preprocessing steps:

    - Tiling Correction: Not for CZI images! Too much work for too little gain
    - Minmax Filtering
    - Outlier removal (i.e. extremely fluorescent imaging artefacts such as    dust particles)
    - Histogram equalization
    - Normalization & minmax rescale

In [12]:
# select raw image file
nb_of_files = len(file_names)
for count, file_name in enumerate(file_names, start = 1):
    image = sdata.images[file_name + '_raw'].data
    nb_channels, y_size, x_size = image.shape

    artefact_masks_array = da.empty_like(image, dtype=bool)
    preprocessed_image = da.empty_like(image, dtype=np.uint16)   

    print(f'Preprocessing: Currently working on {file_name}: {count} of {nb_of_files}')
    
    for ch in range(nb_channels):

        # Tiling Correction
        # Not for SlideScanner images!

        # Minmax filtering
        channel_filter = minimum_filter(image[ch], minmax_filter_size)
        filtered_chanel = image[ch] - maximum_filter(channel_filter, minmax_filter_size)

        # Detect outliers, reduce them and store their segments in SpatialData.labels
        median_value = np.percentile(filtered_chanel.flatten(), 50)
        # Outliers can only be very high => ignore lower-than-median values
        q95 = np.percentile(filtered_chanel[filtered_chanel > median_value].flatten(), 95)[0]
        outliers = filtered_chanel > q95 # define outliers as bigger than 95th quantile
        # Replace outliers with maximally allowed value
        filtered_chanel[outliers] = np.int32(q95)

        # Get larger artefacts from outliers and store them in artefact_masks_array
        larger_artefacts = remove_small_objects_dask(outliers, min_size=artifact_size_threshold)
        artefact_masks_array[ch] = da.asarray(larger_artefacts, dtype=bool)

        # Clahe 
        contr_enhanhanced_image = clahe.apply(filtered_chanel)  #BAD! TODO This reads full array in memory! Make it lazy! 
        
        # Normalize to 0-65535 range
        rescaled_image = minmax_rescaling(contr_enhanhanced_image, feature_range=(0, 65535), dtype=np.uint16)

        # Use dapi channel to create cut mask
        if ch == dapi_ch:
            cut_mask = get_cut_mask(dapi_channel=rescaled_image, median_value=median_value)
            sdata.add_labels(name=file_name + '_cut_mask', labels=Labels2DModel.parse(cut_mask), overwrite=True)

        # Put back in dask array
        preprocessed_image[ch] = da.asarray(rescaled_image) * cut_mask.astype(bool)
        
    # Add to SpatialData object as preprocessed image
    sdata.add_image(name=file_name + '_preprocessed', image=Image2DModel.parse(preprocessed_image), overwrite=True)
    sdata.add_labels(name=file_name + '_artefacts', labels=Labels3DModel.parse(artefact_masks_array), overwrite=True) # 3D because of c-stack

Preprocessing: Currently working on E2M1FloxControl: 1 of 21


KeyboardInterrupt: 